In [1]:
import numpy as np
import pandas as pd

# for time data
import datetime as dt
import pytz

import warnings
warnings.filterwarnings('ignore')

import ast

In [2]:
charging_set = pd.read_csv("data/charging_sessions.csv")

In [3]:
# get overview of data
print("columns:")
print(charging_set.columns)
print("-------------------------------------------------------------------------")
print("info():")
print(charging_set.info())
print("-------------------------------------------------------------------------")
print("describe():")
print(charging_set.describe())

columns:
Index(['Unnamed: 0', 'id', 'connectionTime', 'disconnectTime',
       'doneChargingTime', 'kWhDelivered', 'sessionID', 'siteID', 'spaceID',
       'stationID', 'timezone', 'userID', 'userInputs'],
      dtype='object')
-------------------------------------------------------------------------
info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        66450 non-null  int64  
 1   id                66450 non-null  object 
 2   connectionTime    66450 non-null  object 
 3   disconnectTime    66450 non-null  object 
 4   doneChargingTime  62362 non-null  object 
 5   kWhDelivered      66450 non-null  float64
 6   sessionID         66450 non-null  object 
 7   siteID            66450 non-null  int64  
 8   spaceID           66450 non-null  object 
 9   stationID         66450 non-null  object 
 10  timezone    

In [4]:
charging_set.head(12)

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."
5,5,5e23b149f9af8b5fe4b973d4,2020-01-02 14:00:13+00:00,2020-01-02 23:00:41+00:00,2020-01-02 20:13:21+00:00,13.370,1_1_194_821_2020-01-02 14:00:05.241573,1,AG-1F13,1-1-194-821,America/Los_Angeles,3519.0,"[{'WhPerMile': 600, 'kWhRequested': 24.0, 'mil..."
6,6,5e23b149f9af8b5fe4b973d5,2020-01-02 14:09:14+00:00,2020-01-03 01:57:58+00:00,2020-01-03 00:59:21+00:00,43.477,1_1_178_817_2020-01-02 14:09:14.451827,1,AG-1F09,1-1-178-817,America/Los_Angeles,933.0,"[{'WhPerMile': 385, 'kWhRequested': 65.45, 'mi..."
7,7,5e23b149f9af8b5fe4b973d6,2020-01-02 14:17:32+00:00,2020-01-03 00:44:54+00:00,2020-01-02 20:22:40+00:00,32.441,1_1_191_804_2020-01-02 14:17:32.329535,1,AG-4F34,1-1-191-804,America/Los_Angeles,3737.0,"[{'WhPerMile': 300, 'kWhRequested': 45.0, 'mil..."
8,8,5e23b149f9af8b5fe4b973d7,2020-01-02 14:25:38+00:00,2020-01-02 19:36:05+00:00,2020-01-02 18:38:39+00:00,13.264,1_1_194_826_2020-01-02 14:25:37.578692,1,AG-1F11,1-1-194-826,America/Los_Angeles,419.0,"[{'WhPerMile': 400, 'kWhRequested': 40.0, 'mil..."
9,9,5e23b149f9af8b5fe4b973d8,2020-01-02 14:27:40+00:00,2020-01-02 20:01:48+00:00,2020-01-02 19:17:46+00:00,13.285,1_1_178_823_2020-01-02 14:27:39.582337,1,AG-1F08,1-1-178-823,America/Los_Angeles,651.0,"[{'WhPerMile': 400, 'kWhRequested': 20.0, 'mil..."


# TODO

- check validity and meaning of data (userInput)
- remove irrelevant columns
- split userInput in own columns
- check for duplicates
- check for None-values (userID)
- split userID into binary dummy (isRegistered)
- check for None-values for isRegistered = 1 (remove None-values for registered users)
- check data types of columns
- look at columns with multiple data types (remove wrong entries)
- check ranges of columns & unique values

# Data Cleanup

## Step 1: check validity of columns

- remove unnecessary "unnamed" column

After the first look on the dataset the first column "Unnamed 0" stands out. This column shadows the index and has no value for us. Therefore we remove this column first.

In [5]:
# remove irrelevant columns
charging_set.drop('Unnamed: 0', axis=1, inplace=True)
charging_set.head(3)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."


The second column that instantly stands out is the column "userInputs". This column does not only store one information but a list of different data. Before we can check for duplicates and None-values we have to separate the information stored in "userInputs" and store them in separate columns.

In [6]:
# split userInputs into own columns
def parse_user_inputs(row):
    try:
        # Parse the row as a Python object (e.g., list of dicts)
        parsed = ast.literal_eval(row) if pd.notnull(row) else []
        return parsed[0] if isinstance(parsed, list) and len(parsed) > 0 else {}
    except (ValueError, SyntaxError):
        return {}

# Parse 'userInputs' column in charging_set
charging_set['userInputs'] = charging_set['userInputs'].apply(parse_user_inputs)

# Specify the desired features to extract
desired_features = [
    'WhPerMile', 'kWhRequested', 'milesRequested',
    'minutesAvailable', 'modifiedAt', 'paymentRequired',
    'requestedDeparture'
]

# Extract desired features and create new columns
for feature in desired_features:
    charging_set[feature] = charging_set['userInputs'].apply(
        lambda x: x.get(feature) if isinstance(x, dict) else None
    )
charging_set = charging_set.drop(columns=['userInputs'])
charging_set

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,250.0,25.00,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,280.0,70.00,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,400.0,8.00,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,400.0,8.00,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,400.0,16.00,40.0,504.0,"Thu, 02 Jan 2020 14:00:13 GMT",True,"Thu, 02 Jan 2020 22:24:01 GMT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66445,5d574ad2f9af8b4c10c03652,2019-07-31 18:08:04+00:00,2019-07-31 23:29:18+00:00,2019-07-31 23:30:18+00:00,28.787,1_1_179_809_2019-07-31 18:08:04.432654,1,AG-3F27,1-1-179-809,America/Los_Angeles,393.0,240.0,31.20,130.0,355.0,"Wed, 31 Jul 2019 18:08:23 GMT",True,"Thu, 01 Aug 2019 00:03:04 GMT"
66446,5d574ad2f9af8b4c10c03653,2019-07-31 18:40:41+00:00,2019-08-01 00:59:42+00:00,2019-07-31 21:44:23+00:00,7.787,1_1_179_810_2019-07-31 18:40:40.900203,1,AG-3F30,1-1-179-810,America/Los_Angeles,220.0,333.0,6.66,20.0,455.0,"Wed, 31 Jul 2019 18:41:02 GMT",True,"Thu, 01 Aug 2019 02:15:41 GMT"
66447,5d574ad2f9af8b4c10c03654,2019-07-31 19:04:40+00:00,2019-07-31 22:44:22+00:00,2019-07-31 22:45:21+00:00,11.274,1_1_191_795_2019-07-31 19:04:40.098273,1,AG-4F51,1-1-191-795,America/Los_Angeles,1974.0,333.0,19.98,60.0,184.0,"Wed, 31 Jul 2019 19:04:57 GMT",True,"Wed, 31 Jul 2019 22:08:40 GMT"
66448,5d574ad2f9af8b4c10c03655,2019-07-31 19:19:47+00:00,2019-08-01 00:34:51+00:00,2019-07-31 21:25:30+00:00,11.589,1_1_191_778_2019-07-31 19:19:46.919358,1,AG-4F43,1-1-191-778,America/Los_Angeles,942.0,275.0,22.00,80.0,60.0,"Wed, 31 Jul 2019 19:20:10 GMT",True,"Wed, 31 Jul 2019 20:19:47 GMT"


In [7]:
# check for duplicates
duplicate_counter = charging_set.duplicated().sum()
print("sum of duplicates:", duplicate_counter)
charging_set.drop_duplicates()

sum of duplicates: 1413


,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,250.0,25.00,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,280.0,70.00,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,400.0,8.00,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,400.0,8.00,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,400.0,16.00,40.0,504.0,"Thu, 02 Jan 2020 14:00:13 GMT",True,"Thu, 02 Jan 2020 22:24:01 GMT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65032,5d2fbdd3f9af8b4d0dd0d54a,2019-07-01 21:49:12+00:00,2019-07-02 01:37:18+00:00,2019-07-01 23:14:33+00:00,8.399,1_1_179_798_2019-07-01 21:49:11.873404,1,AG-3F18,1-1-179-798,America/Los_Angeles,1346.0,400.0,32.00,80.0,36.0,"Mon, 01 Jul 2019 21:49:37 GMT",True,"Mon, 01 Jul 2019 22:25:12 GMT"
65033,5d2fbdd3f9af8b4d0dd0d54b,2019-07-01 21:58:45+00:00,2019-07-02 00:39:48+00:00,2019-07-02 00:40:21+00:00,16.864,1_1_179_794_2019-07-01 21:58:44.571011,1,AG-3F20,1-1-179-794,America/Los_Angeles,364.0,400.0,40.00,100.0,234.0,"Mon, 01 Jul 2019 21:59:09 GMT",True,"Tue, 02 Jul 2019 01:52:45 GMT"
65034,5d2fbdd3f9af8b4d0dd0d54c,2019-07-01 22:02:21+00:00,2019-07-02 00:58:50+00:00,2019-07-02 00:59:23+00:00,18.335,1_1_191_807_2019-07-01 22:02:20.810735,1,AG-4F47,1-1-191-807,America/Los_Angeles,2050.0,333.0,29.97,90.0,309.0,"Mon, 01 Jul 2019 22:02:37 GMT",True,"Tue, 02 Jul 2019 03:11:21 GMT"
65035,5d2fbdd3f9af8b4d0dd0d54d,2019-07-01 22:23:44+00:00,2019-07-02 02:03:36+00:00,2019-07-02 02:04:06+00:00,22.815,1_1_179_781_2019-07-01 22:23:32.496137,1,AG-3F31,1-1-179-781,America/Los_Angeles,1626.0,200.0,34.00,170.0,99.0,"Mon, 01 Jul 2019 22:23:43 GMT",True,"Tue, 02 Jul 2019 00:02:44 GMT"


In [8]:
# check for none-values
nan_counts = charging_set.isna().sum()
nan_counts_sorted = nan_counts.sort_values(ascending=False)

for column, count in nan_counts_sorted.items():
    print(f"Feature: {column}, Count of NaN-Values: {count}")

Feature: paymentRequired, Count of NaN-Values: 17263
Feature: requestedDeparture, Count of NaN-Values: 17263
Feature: milesRequested, Count of NaN-Values: 17263
Feature: kWhRequested, Count of NaN-Values: 17263
Feature: modifiedAt, Count of NaN-Values: 17263
Feature: minutesAvailable, Count of NaN-Values: 17263
Feature: WhPerMile, Count of NaN-Values: 17263
Feature: userID, Count of NaN-Values: 17263
Feature: doneChargingTime, Count of NaN-Values: 4088
Feature: id, Count of NaN-Values: 0
Feature: connectionTime, Count of NaN-Values: 0
Feature: timezone, Count of NaN-Values: 0
Feature: spaceID, Count of NaN-Values: 0
Feature: stationID, Count of NaN-Values: 0
Feature: sessionID, Count of NaN-Values: 0
Feature: kWhDelivered, Count of NaN-Values: 0
Feature: disconnectTime, Count of NaN-Values: 0
Feature: siteID, Count of NaN-Values: 0


We observe a large number of NaN-values in the columns that were separated from the column \"userInputs\". It is very noticeable that the number of NaN-values of the \"userInputs\"-columns is the same as the number of the NaN-values in the \"userID\"-column. Accordingly, we assume that the NaN-values in the \"userInputs\"-columns originate from users not being registered. In the following, we check if our assumption is correct:

In [9]:
# list of userInputs columns
user_inputs = ["requestedDeparture", "milesRequested", "paymentRequired", "modifiedAt", "minutesAvailable", "WhPerMile", "kWhRequested"]

def check_user_inputs_nan_condition(user_inputs_columns):
    """
    check_user_inputs_nan_condition checks if userID is NaN when userInputs columns are NaN.
    
    :param user_inputs_columns: list of user input columns that were stored in the former "userInputs"-column
    :return: True if no violations are found, False if a userID exists with NaN in the corresponding userInput column.
    """
    # iterate over each row and compare userID-value to userInputs-values
    for idx, row in charging_set.iterrows():
        user_id_value = row["userID"]

        for user_input_column in user_inputs_columns:
            user_input_value = row[user_input_column]

            # if userID is not NaN but the userInputs-column is NaN, return false
            if pd.isna(user_input_value) and not pd.isna(user_id_value):
                return False

    # if no violation of condition is found, return true
    return True

# check if unregistered users are the reason for NaN-values in userInputs columns
print("Unregistered users are the reason for NaN-values in userInputs columns:", check_user_inputs_nan_condition(user_inputs))

# check if we can use -1 as a placeholder for NaN values for "userID" and userInputs columns, except for the time data since we want to treat them differently
columns_contain_nan = ["userID", "milesRequested", "paymentRequired", "minutesAvailable", "WhPerMile", "kWhRequested"]

contains_zero_value = (charging_set[columns_contain_nan] == -1).any()

print("userID contains a -1 value:\n", contains_zero_value)

Unregistered users are the reason for NaN-values in userInputs columns: True
userID contains a -1 value:
 userID              False
milesRequested      False
paymentRequired     False
minutesAvailable    False
WhPerMile           False
kWhRequested        False
dtype: bool


In [10]:
# fill NaN values with -1 in columns that do not contain zero values, for the rest of the columns, use -1
charging_set[columns_contain_nan] = charging_set[columns_contain_nan].fillna(-1)

Additionally, we create a new column called isRegistered, since the exact ID of a user is not relevant, but only the information wether a user is registered or not.

In [11]:
# isRegistered should be 0 if a user is not registered (userID == NaN) and be 1 if a user is registered
charging_set['isRegistered'] = charging_set['userID'].notnull().astype(int)

# Time-Data

Now we have a look at the time data. Just as we filled the NaN-values for the userInputs with -1 as placeholders, we will use the minimum possible value for python's "datetime" datatype (01.01.0001, midnight UTC) as an equivalent to the 0 value we used for the other numeric columns. After, we will convert the time-data into human-readable format.

In [12]:
# convert time-data in correct datatype
charging_set['connectionTime'] = pd.to_datetime(charging_set['connectionTime'])
charging_set['disconnectTime'] = pd.to_datetime(charging_set['disconnectTime'])
charging_set['doneChargingTime'] = pd.to_datetime(charging_set['doneChargingTime'])
charging_set['modifiedAt'] = pd.to_datetime(charging_set['modifiedAt'])
charging_set['requestedDeparture'] = pd.to_datetime(charging_set['requestedDeparture'])

charging_set['connectionTime_year'] = charging_set['connectionTime'].dt.year
charging_set['connectionTime_month'] = charging_set['connectionTime'].dt.month
charging_set['connectionTime_day'] = charging_set['connectionTime'].dt.day
charging_set['connectionTime_time'] = charging_set['connectionTime'].dt.time
charging_set['connectionTime_weekday'] = charging_set['connectionTime'].dt.weekday

charging_set['disconnectTime_year'] = charging_set['disconnectTime'].dt.year
charging_set['disconnectTime_month'] = charging_set['disconnectTime'].dt.month
charging_set['disconnectTime_day'] = charging_set['disconnectTime'].dt.day
charging_set['disconnectTime_time'] = charging_set['disconnectTime'].dt.time
charging_set['disconnectTime_weekday'] = charging_set['disconnectTime'].dt.weekday

charging_set['doneChargingTime_year'] = charging_set['doneChargingTime'].dt.year
charging_set['doneChargingTime_month'] = charging_set['doneChargingTime'].dt.month
charging_set['doneChargingTime_day'] = charging_set['doneChargingTime'].dt.day
charging_set['doneChargingTime_time'] = charging_set['doneChargingTime'].dt.time
charging_set['doneChargingTime_weekday'] = charging_set['doneChargingTime'].dt.weekday

charging_set['modifiedAt_year'] = charging_set['modifiedAt'].dt.year
charging_set['modifiedAt_month'] = charging_set['modifiedAt'].dt.month
charging_set['modifiedAt_day'] = charging_set['modifiedAt'].dt.day
charging_set['modifiedAt_time'] = charging_set['modifiedAt'].dt.time
charging_set['modifiedAt_weekday'] = charging_set['modifiedAt'].dt.weekday

charging_set['requestedDeparture_year'] = charging_set['requestedDeparture'].dt.year
charging_set['requestedDeparture_month'] = charging_set['requestedDeparture'].dt.month
charging_set['requestedDeparture_day'] = charging_set['requestedDeparture'].dt.day
charging_set['requestedDeparture_time'] = charging_set['requestedDeparture'].dt.time
charging_set['requestedDeparture_weekday'] = charging_set['requestedDeparture'].dt.weekday

In [13]:
def calculate_hours():
    # Berechne die Ladezeit als Differenz zwischen disconnectTime und connectionTime
    charging_set["chargingDurationTemp"] = charging_set["disconnectTime"] - charging_set["connectionTime"]
    
    # Konvertiere die Differenz in Stunden
    charging_set['chargingDuration'] = charging_set["chargingDurationTemp"].apply(
        lambda x: x.total_seconds() / 3600
    )
    
    # Entferne die temporäre Spalte
    charging_set.drop(columns=["chargingDurationTemp"], inplace=True)
    
    return charging_set

# Aufruf der Funktion
charging_set = calculate_hours()

In [14]:
### check data types of each column
# Find the unique types
# TODO: add userInputs columns into here
id_types = charging_set["id"].apply(type).unique()
connectionTime_types = charging_set["connectionTime"].apply(type).unique()
disconnectTime_types = charging_set["disconnectTime"].apply(type).unique()
doneChargingTime_types = charging_set["doneChargingTime"].apply(type).unique()
kWhDelivered_types = charging_set["kWhDelivered"].apply(type).unique()
siteID_types = charging_set["siteID"].apply(type).unique()
spaceID_types = charging_set["spaceID"].apply(type).unique()
stationID_types = charging_set["stationID"].apply(type).unique()
timezone_types = charging_set["timezone"].apply(type).unique()
userID_types = charging_set["userID"].apply(type).unique()

# Print the unique types
print("id types:", id_types)
print("connectionTime types:", connectionTime_types)
print("disconnectTime types:", disconnectTime_types)
print("doneChargingTime types:", doneChargingTime_types)
print("kWhDelivered types:", kWhDelivered_types)
print("siteID types:", siteID_types)
print("spaceID types:", spaceID_types)
print("stationID types:", stationID_types)
print("timezone types:", timezone_types)
print("userID types:", userID_types)

id types: [<class 'str'>]
connectionTime types: [<class 'pandas._libs.tslibs.timestamps.Timestamp'>]
disconnectTime types: [<class 'pandas._libs.tslibs.timestamps.Timestamp'>]
doneChargingTime types: [<class 'pandas._libs.tslibs.timestamps.Timestamp'>
 <class 'pandas._libs.tslibs.nattype.NaTType'>]
kWhDelivered types: [<class 'float'>]
siteID types: [<class 'int'>]
spaceID types: [<class 'str'>]
stationID types: [<class 'str'>]
timezone types: [<class 'str'>]
userID types: [<class 'float'>]


In [15]:
# Check ranges of numeric columns
numeric_columns = charging_set.select_dtypes(include=[np.number])
for col in numeric_columns.columns:
    print(f"Column '{col}': min={numeric_columns[col].min()}, max={numeric_columns[col].max()}")

# Check unique values of categorical columns
categorical_columns = charging_set.select_dtypes(exclude=[np.number])
for col in categorical_columns.columns:
    print(f"Column '{col}': unique values={categorical_columns[col].unique()}")

Column 'kWhDelivered': min=0.501, max=108.79724166666666
Column 'siteID': min=1, max=2
Column 'userID': min=-1.0, max=19923.0
Column 'WhPerMile': min=-1.0, max=2000.0
Column 'kWhRequested': min=-1.0, max=215.32
Column 'milesRequested': min=-1.0, max=775.0
Column 'minutesAvailable': min=-1.0, max=10062.0
Column 'isRegistered': min=1, max=1
Column 'connectionTime_year': min=2018, max=2021
Column 'connectionTime_month': min=1, max=12
Column 'connectionTime_day': min=1, max=31
Column 'connectionTime_weekday': min=0, max=6
Column 'disconnectTime_year': min=2018, max=2021
Column 'disconnectTime_month': min=1, max=12
Column 'disconnectTime_day': min=1, max=31
Column 'disconnectTime_weekday': min=0, max=6
Column 'doneChargingTime_year': min=2018.0, max=2021.0
Column 'doneChargingTime_month': min=1.0, max=12.0
Column 'doneChargingTime_day': min=1.0, max=31.0
Column 'doneChargingTime_weekday': min=0.0, max=6.0
Column 'modifiedAt_year': min=2018.0, max=2021.0
Column 'modifiedAt_month': min=1.0, m

The timezone column only has one value: "". Hence, we can remove it since it does not bring any new information to the analysis.

In [16]:
charging_set.drop('timezone', axis=1, inplace=True)
charging_set.head(3)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,userID,...,modifiedAt_month,modifiedAt_day,modifiedAt_time,modifiedAt_weekday,requestedDeparture_year,requestedDeparture_month,requestedDeparture_day,requestedDeparture_time,requestedDeparture_weekday,chargingDuration
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,194.0,...,1.0,2.0,13:09:39,3.0,2020.0,1.0,2.0,20:51:54,3.0,6.039167
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,4275.0,...,1.0,2.0,13:37:11,3.0,2020.0,1.0,2.0,23:31:50,3.0,9.025278
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,344.0,...,1.0,2.0,13:57:17,3.0,2020.0,1.0,2.0,14:56:35,3.0,10.713056


In [17]:
charging_set.to_csv('data/cleaned_charging_set.csv', index=False)